# Predicitng tasks from input data

## importing necessary modules

In [2]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
import importlib
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import optuna
import imaplib


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score


#importing fundtion to get encoder info and perfrom tasks 

from get_finetune_encoder import get_finetune_encoder_from_modelID, get_input_data
from freez_encoder import fine_tune_model


import freez_encoder
importlib.reload(freez_encoder)


%who



[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
2024-09-03 02:14:35.001278: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-03 02:14:35.032973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 02:14:35.071156: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 02:14:35.082805: E external/local_xla/xla/stream_executor/c

DataLoader	 LabelEncoder	 TensorDataset	 accuracy_score	 f1_score	 fine_tune_model	 finetune_save_dir	 freez_encoder	 get_finetune_encoder_from_modelID	 
get_input_data	 imaplib	 importlib	 input_data_location	 label_encoder	 ml_code_path	 nn	 np	 optim	 
optuna	 os	 pd	 precision_score	 random	 recall_score	 roc_auc_score	 task_list_cat	 task_list_survival	 
torch	 


## importing input data

In [3]:
#input data
input_data_location='/home/leilapirhaji/PROCESSED_DATA_2'
finetune_save_dir='/home/leilapirhaji/finetune_frozen_layer' 


#tasks to predict using encoder
task_list_cat=['Benefit BINARY', 'Nivo Benefit BINARY', 'MSKCC BINARY', 'IMDC BINARY', 'Benefit ORDINAL', 'MSKCC ORDINAL', 'IMDC ORDINAL', 'ORR', 'Benefit', 'IMDC', 'MSKCC', 'Prior_2' ]

#survival tasks
task_list_survival=[ 'OS', 'OS_Event', 'PFS', 'PFS_Event' ]


#get fine-tuning input data 
(X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=get_input_data(input_data_location)

X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape

((443, 2736), (443, 58), (149, 2736), (149, 58), (149, 2736), (149, 58))

## Predicting binary tasks

#### IMDC BInary

In [6]:
import latent_task_predict
from latent_task_predict import log_reg_multi_class
import importlib

importlib.reload(latent_task_predict)

task = 'IMDC BINARY'

# Filter out rows in y_train, y_val, y_test with NA values
y_train_filtered = y_data_train[task].dropna()
x_train_filtered = X_data_train.loc[y_train_filtered.index]

y_val_filtered = y_data_val[task].dropna()
x_val_filtered = X_data_val.loc[y_val_filtered.index]

y_test_filtered = y_data_test[task].dropna()
x_test_filtered = X_data_test.loc[y_test_filtered.index]

y_train_filtered.shape, x_train_filtered.shape, y_val_filtered.shape, x_val_filtered.shape, y_test_filtered.shape, x_test_filtered.shape

#Pass the filtered data to the model
best_model, val_accuracy, test_accuracy = log_reg_multi_class(
    x_train_filtered, 
    y_train_filtered, 
    x_val_filtered, 
    y_val_filtered, 
    x_test_filtered, 
    y_test_filtered
)

val_accuracy, test_accuracy

(0.8688524590163934, 0.8070175438596491)

## Predicting multi-class tasks 

## Predicting Survival task

### OS prediction

In [10]:
import importlib

import latent_task_predict_finetune
importlib.reload(latent_task_predict_finetune)
from latent_task_predict_finetune import cox_proportional_hazards


task = 'OS'
task_event = 'OS_Event'


#Pass the filtered data to the model
(best_val_c_index, best_test_c_index, best_params)= cox_proportional_hazards(X_data_train, y_data_train[task], y_data_train[task_event], X_data_val, y_data_val[task], y_data_val[task_event], X_data_test, y_data_test[task], y_data_test[task_event])

best_val_c_index, best_test_c_index, best_params

(0.715054309207187, 0.6761040773601834, {'penalizer': 10, 'l1_ratio': 0})

### PFS prediction 